# Writer Monad

## Outline

* Incentive for the Writer monad

* The Writer monad
  - Definition of the Writer monad
  - MonadWriter type class
  - Simple example
<br><br>
* Writer monad examples

In this lesson, we will learn about the Writer monad type and how you can use it.

## Incentive for Writer monad

In the previous lecture we learned that with the Reader monad we can more easily pass data around by accessing the environment variable of the Reader monad. 

But what if we need to modify this data? Let's look at an example where you process a string depending on it's length and count the operations peformed on the string.

In [ ]:
type OperationsCount = Int

check1 :: String -> (OperationsCount, String)
check1 input = if length input < 5
  then check2 (1, input ++ "_below5")
  else check3 (1, input ++ "_above4")

check2 :: (OperationsCount, String) -> (OperationsCount, String)
check2 (operations, input) = if length input < 10
  then (operations, input)
  else (operations + 1, input ++ "_above9")

check3 :: (OperationsCount, String) -> (OperationsCount, String)
check3 (operations, input) = if length input < 16
  then check2 (operations + 1, removeFirst4 input)
  else check4 (operations + 1, input ++ "_above15")
  where removeFirst4 xs = [(!! id) xs | id <- [3..((length xs) - 1)]]

check4 :: (OperationsCount, String) -> (OperationsCount, String)
check4 (operations, input) = if length input < 30
  then (operations + 1 , input ++ "_above29")
  else (operations + 1, input ++ "_below30")

main :: IO ()
main = do
  let (count1, result1) = check1 "test"
  print "Processing word: test"
  print $ "Operations performed: " ++ show count1
  print $ "Final result: " ++ result1
  print "------------------------"
  let (count2, result2) = check1 "testing"
  print "Processing word: testing"
  print $ "Operations performed: " ++ show count2
  print $ "Final result: " ++ result2

main

Passing data around and modifying it can get tedious when programs get bigger. 

For this reason the **Writer monad** was created that allows you to pass data and update it between different Writer monads. 

## The Writer monad

### Definition of the Writer monad

Let's first look at the user freindly definition of **Writer** type and then the actuall Haskell2010 definition that uses transformers.

The definition of the Writer type can be written as:
```haskell
newtype Writer w a = Writer {runWriter :: (a,w)}
```

We see that the `Writer` data constructor holds a variable that can be accessed with the name `runWriter`.

The variable is a tuple of type `(a,w)`.

We can now create a Monad instance for `Writer w` and not just `Writer`.

This means the type of our writing variable will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: Writer w a        ->
         (a -> Writer w b) ->
         Writer w b
```

The monad instance for `Writer w` would be:
```haskell
instance (Monoid w) => Monad (Writer w) where
    return a             = Writer (a,mempty)
    (Writer (a,w)) >>= f = let (a',w') = runWriter $ f a 
                           in Writer (a',w `mappend` w')
```

The actual Haskell2010 definition of the `Writer` type is defined in terms of the `WriterT` monad transformer:
```haskell
type Writer w = WriterT w Identity
```
We will learn more about it when talk about Monad transformers.

### MonadWriter type class

The `MonadWriter` type class defines functions wich you can use to work with a writer monad. It is defined in the **Control.Monad.Writer** module.

The `tell` function appends data to the writer variable.

The `listen` function retrieves the writer variable and the variable of type `a` from the Writer monad.

The `pass` function takes in a writer monad that holds a function which then gets applied to the writer variable and the updated writer monad is returned. 

```haskell
class (Monoid w, Monad m) => MonadWriter w m | m -> w where
    writer :: (a,w) -> m a
    writer ~(a, w) = do
      tell w
      return a

    tell   :: w -> m ()
    tell w = writer ((),w)

    listen :: m a -> m (a, w)

    pass   :: m (a, w -> w) -> m a
    {-# MINIMAL (writer | tell), listen, pass #-}
```

To be able to declare a type class with two types you need to use the *MultiParamTypeClasses* language pragma.

If the type `m` which in reality for us will be `(Writer w)` is defiinng the type `w` we can state this dependency with the `| m -> r` statement.

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma.

For the `Writer w` monad the instance of the `MonadWriter` type class would be:
```haskell
instance (Monoid w) => MonadWriter w (Writer w) where
    pass   (Writer ((a,f),w)) = Writer (a,f w)
    listen (Writer (a,w))     = Writer ((a,w),w)
    tell   s                  = Writer ((),s)
```

Form the module that contains the `ReaderWriter` type class we can also use the `listens` and `censor` functions.
```haskell
listens :: MonadWriter w m => (w -> b) -> m a -> m (a, b)
listens f m = do
    ~(a, w) <- listen m
    return (a, f w)

censor :: MonadWriter w m => (w -> w) -> m a -> m a
censor f m = pass $ do
    a <- m
    return (a, f)
```

`listens` takes in a function and a writer monad. Then it applies the function to the writer variable and returns the updated writer monad.

`censor` takes in a function and a writer monad and applies this function to the writer variable to update the writer monad.

### Simple example

Here is a simple Writer monad example that uses the `tell` and `runWriter` functions.

In [ ]:
import Control.Monad.Writer

logAge :: Int -> Writer [String] Int  
logAge x = do
    tell ["Age is: " ++ show x]
    return x            
      
sumAge :: Writer [String] Int  
sumAge = do  
    a <- logAge 16  
    b <- logAge 18
    tell ["Summed age is: " ++ (show $ a + b)]
    return (a + b)

main :: IO ()
main = do
    let (age, messages) = runWriter sumAge
    mapM_ putStrLn messages

main

We see that every time we use the `tell` function the value gets appended to the first parameter of the Writer monad. 

The type of the parameter which is in our case a list has to have an instance of the **Monoid** type class, which lists have.

## Writer monad examples

Let's implement the example from the first chapter with the use of the Writer monad.

In [ ]:
instance Semigroup Int where
  a <> b = a + b

instance Monoid Int where
  mempty = 0

check1' :: String -> (Int, String)
check1' input = if length input < 5
  then swap $ runWriter $ do
    tell 1
    check2' $ input ++ "_below5"
  else swap $ runWriter $ do
    tell 1
    check3' $ input ++ "_above4"
  where swap (x,y) = (y,x)

check2' :: String -> Writer Int String
check2' input = if length input < 10
  then do
    return input
  else do
    tell 1
    return $ input ++ "_above9"

check3' :: String -> Writer Int String
check3' input = if length input < 16
  then do
    tell 1
    check2' $ removeFirst4 input
  else do
    tell 1
    check4' $ input ++ "_above15"
  where removeFirst4 xs = [(!! id) xs | id <- [3..((length xs) - 1)]]

check4' :: String -> Writer Int String
check4' input = if length input < 10
  then do
    tell 1
    return $ input ++ "_above29"
  else do
    tell 1
    return $ input ++ "_below30"

main2 :: IO ()
main2 = do
  let (count1, result1) = check1' "test"
  print "Processing word: test"
  print $ "Operations performed: " ++ show count1
  print $ "Final result: " ++ result1
  print "------------------------"
  let (count2, result2) = check1' "testing"
  print "Processing word: testing"
  print $ "Operations performed: " ++ show count2
  print $ "Final result: " ++ result2

main2

Since the type of the write parameter has to have an instance of Monoid we implement it in the begining for `Int`. 

After that we define the check functions similar as in the previous example where we use the `tell` function to keep track of the operations we have performed. 

Here is another example where we use also the `listen` function that enables us to retrieve the logs from the Writer monad.

In [ ]:
type Adding a = Writer [String] a

logMsg :: String -> Adding ()
logMsg msg = tell [msg]

add1 :: Int -> Adding Int
add1 x = do
    logMsg "Starting add1."
    let y = x + 1
    logMsg $ "Computed result: " ++ show y
    return y

start :: Int -> Adding Int
start x = do
    (n, logs) <- listen $ add1 x
    let logLines = length logs
    logMsg $ "add1 logged " ++ show logLines ++ " lines"
    return $ n

main :: IO ()
main = do
    print "Input an integer number:"
    n <- (read <$> getLine) :: IO Int
    let (result, logs) = runWriter $ start n
    print $ "Result is: " ++ show result
    putStrLn "Logs are: "
    mapM_ print logs

main

In the final example we demonstrate how the `pass` function works. We use the code from the simple example chapter that we modify a bit.

In [ ]:
logAge :: Int -> Writer [String] Int  
logAge x = do
    tell ["Age is: " ++ show x]
    return x            
      
sumAge :: Writer [String] (Int, [String] -> [String])
sumAge = do  
    a <- logAge 16  
    b <- logAge 18
    tell ["Summed age is: " ++ (show $ a + b)]
    return (a + b,transform)

process :: Writer [String] Int
process = do
    pass sumAge

transform :: [String] -> [String]
transform list = map ("NOTE: " ++) list

main :: IO ()
main = do
    let (age, messages) = runWriter process
    mapM_ putStrLn 
    
main

Note how we also had to change the type signature of the `sumAge` writer monad to include the `transform` functions type signature.

## Recap

In this lesson we've discussed:

- the motivation for introducing the Writer monad type 

- the definition of the Writer monad type and a simple example

- examples that use the Writer monad type to solve problems